# Master Thesis - Simulated Pick and Place Machine - Data Gathering

In [8]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import pandas as pd
import numpy as np

def convert_to_entry(input_dict):
    return {key: [value] for key, value in input_dict.items()}

def convert_list_df(input_list):
    df = pd.DataFrame(convert_to_entry(input_list[0]))
    for i in range(1,len(input_list)):
        df.loc[i] = convert_to_entry(input_list[i])
    return df


## Connection to Simulation

In [9]:
client = RemoteAPIClient()


sim = client.require('sim')
simBWF = client.require('simBWF')

sim.setStepping(True)

sim.startSimulation()

camera_1 = sim.getObject('/camera_1/camera')
camera_4 = sim.getObject('/camera_4/camera')
camera_1_df = pd.DataFrame()
camera_4_df = pd.DataFrame()

conveyor1 = sim.getObject("/genericConveyorTypeA[0]")
conveyor2 = sim.getObject("/genericConveyorTypeA[2]")
conveyor3 = sim.getObject("/genericConveyorTypeA[1]")
conveyor_1_df = pd.DataFrame()
conveyor_2_df = pd.DataFrame()
conveyor_3_df = pd.DataFrame()

detectionWindow_1 = sim.getObject("/genericDetectionWindow[3]")
detection_1_df = pd.DataFrame()



while (t := sim.getSimulationTime()) < 10:
    camera_1_data = pd.DataFrame([sim.unpackTable(sim.readCustomDataBlock(camera_1,'customData'))])
    camera_4_data = pd.DataFrame([sim.unpackTable(sim.readCustomDataBlock(camera_4,'customData'))])

    conveyor_1_data = pd.DataFrame([sim.unpackTable(sim.readCustomDataBlock(conveyor1, simBWF.modelTags.CONVEYOR))])
    conveyor_2_data = pd.DataFrame([sim.unpackTable(sim.readCustomDataBlock(conveyor2, simBWF.modelTags.CONVEYOR))])
    conveyor_3_data = pd.DataFrame([sim.unpackTable(sim.readCustomDataBlock(conveyor3, simBWF.modelTags.CONVEYOR))])

    detection_1_data = pd.DataFrame([sim.unpackTable(sim.readCustomDataBlock(detectionWindow_1,'customData'))])

    camera_1_df= pd.concat([camera_1_df,camera_1_data], ignore_index=True)
    camera_4_df= pd.concat([camera_4_df,camera_4_data], ignore_index=True)

    conveyor_1_df= pd.concat([conveyor_1_df,conveyor_1_data['velocity']], ignore_index=True)
    conveyor_2_df= pd.concat([conveyor_2_df,conveyor_2_data['velocity']], ignore_index=True)
    conveyor_3_df= pd.concat([conveyor_3_df,conveyor_3_data['velocity']], ignore_index=True)

    detection_1_df= pd.concat([detection_1_df,detection_1_data], ignore_index=True)

    sim.step()

sim.stopSimulation()

## Data preparation

In [10]:
camera_1_df.replace(np.nan,0,inplace=True)
camera_4_df.replace(np.nan,0,inplace=True)

conveyor_1_df.replace(np.nan,0,inplace=True)
conveyor_2_df.replace(np.nan,0,inplace=True)
conveyor_3_df.replace(np.nan,0,inplace=True)

detection_1_df.replace(np.nan,0,inplace=True)
